<a href="https://colab.research.google.com/github/GeorgeKonstantinosDimou/Skroutz/blob/main/thesisKGE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing

In [1]:
import pandas as pd
import io
from tqdm import tqdm
from google.colab import files
from IPython.display import Markdown
!pip install torchkge
import torchkge

from torch import cuda
from torch.optim import SGD, Adam


#from torchkge.models.interfaces.TranslationModel import TransH, TransD, TorusE
#from torchkge.evaluation import TripletClassificationEvaluator

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.6/51.6 KB 2.1 MB/s eta 0:00:00


/usr/local/lib/python3.9/dist-packages/torchkge/utils/data_redundancy.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


#Read csv

In [2]:
uploaded = files.upload()

Saving expTriplets.json to expTriplets.json


In [3]:
df = pd.read_json(io.BytesIO(uploaded['expTriplets.json']), lines=True)
print(df.dtypes)

#df.astype({'to': 'str'}).dtypes
df['to'] = df['to'].apply(str)

from    object
to      object
rel     object
dtype: object


#Training

In [4]:
def train_test_kge(parameters: dict):

    kg_train, kg_val, kg_test = torchkge.data_structures.KnowledgeGraph(df).split_kg(validation=True)

    model = torchkge.models.translation.TransHModel(parameters['emb_size'], kg_train.n_ent, kg_train.n_rel)
    #model = torchkge.models.translation.TransDModel(parameters['ent_emb_size'], parameters['rel_emb_size'], kg_train.n_ent, kg_train.n_rel)
    criterion = torchkge.utils.BinaryCrossEntropyLoss()

    if cuda.is_available():
        cuda.empty_cache()
        model.cuda()
        criterion.cuda()

    optimizer = Adam(model.parameters(), lr=parameters['lr'], weight_decay=1e-5)
    #optimizer = SGD(model.parameters(), lr=parameters['lr'], momentum=0.9)
    sampler = torchkge.sampling.BernoulliNegativeSampler(kg_train)
    dataloader = torchkge.utils.DataLoader(kg_train, batch_size=parameters['batch_size'], use_cuda='all')
    #dataloader = torchkge.utils.training.TrainDataLoader(kg_train, batch_size=parameters['batch_size'], use_cuda='all')

    iterator = tqdm(range(parameters['n_epochs']), unit='epoch')
    for epoch in iterator:
        running_loss = 0.0
        for i, batch in enumerate(dataloader):
            h, t, r = batch[0], batch[1], batch[2]
            n_h, n_t = sampler.corrupt_batch(h, t, r)

            optimizer.zero_grad()

            # forward + backward + optimize
            pos, neg = model(h, t, r, n_h, n_t)
            loss = criterion(pos, neg)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        iterator.set_description(
            'Epoch {} | mean loss: {:.5f}'.format(epoch + 1,
                                                running_loss / len(dataloader)))
        model.normalize_parameters()

    model.normalize_parameters()

    #Testing
    evaluator = torchkge.evaluation.TripletClassificationEvaluator(model, kg_val, kg_test)
    evaluator.evaluate(parameters['batch_size'])
    print('Accuracy on test set: {}'.format(evaluator.accuracy(parameters['batch_size'])))

In [5]:
def main():

    hyperparameters = {
        'emb_size': 100,
        'ent_emb_size': 100,
        'rel_emb_size' : 100,
        'lr': 0.0001,
        'n_epochs': 40,
        'batch_size': 2500
    }

    train_test_kge(hyperparameters)

if __name__=='__main__':
    main()

Epoch 40 | mean loss: 2178.52234: 100%|██████████| 40/40 [11:58<00:00, 17.97s/epoch]


RuntimeError: ignored